
# Revenue Analytics using Window Functions (MySQL)

The dataset represents an e-commerce system with:
- orders
- order_items



- orders(order_id, order_date, order_status)
- order_items(order_item_order_id, order_item_product_id, order_item_subtotal)




## Daily Revenue

Compute total revenue per day for completed and closed orders.


In [ ]:

CREATE TABLE daily_revenue AS
SELECT 
    o.order_date,
    ROUND(SUM(oi.order_item_subtotal), 2) AS order_revenue
FROM orders o
JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date;



View daily revenue ordered by date.


In [ ]:

SELECT *
FROM daily_revenue
ORDER BY order_date;



## Daily Product Revenue

Compute revenue per product per day.


In [ ]:

CREATE TABLE daily_product_revenue AS
SELECT 
    o.order_date,
    oi.order_item_product_id,
    ROUND(SUM(oi.order_item_subtotal), 2) AS order_revenue
FROM orders o
JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date, oi.order_item_product_id;



View daily product revenue sorted by date and revenue.


In [ ]:

SELECT *
FROM daily_product_revenue
ORDER BY order_date, order_revenue DESC;



## Monthly Revenue using Window Functions

Compute monthly revenue while retaining daily rows.


In [ ]:

SELECT 
    DATE_FORMAT(order_date, '%Y-%m') AS order_month,
    order_date,
    order_revenue,
    SUM(order_revenue) OVER (
        PARTITION BY DATE_FORMAT(order_date, '%Y-%m')
    ) AS monthly_order_revenue
FROM daily_revenue
ORDER BY order_date;



## Total Revenue using Window Function

Compute total revenue across all days without aggregation.


In [ ]:

SELECT 
    dr.*,
    SUM(order_revenue) OVER () AS total_order_revenue
FROM daily_revenue dr
ORDER BY order_date;



## Basic Exploration

Count records and inspect sorted data.


In [ ]:

SELECT COUNT(*) FROM daily_product_revenue;

SELECT *
FROM daily_product_revenue
ORDER BY order_date, order_revenue DESC;



## Global Ranking using RANK and DENSE_RANK

Rank products globally by revenue for a given date.


In [ ]:

SELECT 
    order_date,
    order_item_product_id,
    order_revenue,
    RANK() OVER (ORDER BY order_revenue DESC) AS rnk,
    DENSE_RANK() OVER (ORDER BY order_revenue DESC) AS drnk
FROM daily_product_revenue
WHERE order_date = '2014-01-01'
ORDER BY order_revenue DESC;



## Ranking Within Each Day

Rank products within each order_date.


In [ ]:

SELECT 
    order_date,
    order_item_product_id,
    order_revenue,
    RANK() OVER (
        PARTITION BY order_date
        ORDER BY order_revenue DESC
    ) AS rnk,
    DENSE_RANK() OVER (
        PARTITION BY order_date
        ORDER BY order_revenue DESC
    ) AS drnk
FROM daily_product_revenue
WHERE DATE_FORMAT(order_date, '%Y-%m') = '2014-01'
ORDER BY order_date, order_revenue DESC;



## Filtering Top-N Products using Nested Query

Retrieve top 5 products by revenue using global ranking.


In [ ]:

SELECT *
FROM (
    SELECT 
        order_date,
        order_item_product_id,
        order_revenue,
        RANK() OVER (ORDER BY order_revenue DESC) AS rnk,
        DENSE_RANK() OVER (ORDER BY order_revenue DESC) AS drnk
    FROM daily_product_revenue
    WHERE order_date = '2014-01-01'
) ranked
WHERE drnk <= 5
ORDER BY order_revenue DESC;



## Filtering Top-N Products using CTE

Same logic written using a Common Table Expression.


In [ ]:

WITH daily_product_revenue_ranks AS (
    SELECT 
        order_date,
        order_item_product_id,
        order_revenue,
        RANK() OVER (ORDER BY order_revenue DESC) AS rnk,
        DENSE_RANK() OVER (ORDER BY order_revenue DESC) AS drnk
    FROM daily_product_revenue
    WHERE order_date = '2014-01-01'
)
SELECT *
FROM daily_product_revenue_ranks
WHERE drnk <= 5
ORDER BY order_revenue DESC;
